**DATA CLEANING ON NEW YORK CITY AIRBNB LISTINGS**





In [3]:
import pandas as pd
from scipy.stats import zscore
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
import string

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anike\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anike\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\anike\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
df=pd.read_csv("AB_NYC_2019.csv")

In [9]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [10]:
df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [11]:
df.shape

(48895, 16)

In [12]:
df.isnull().sum()

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [13]:
df.isna().sum()

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

In [15]:
df.duplicated().sum()

0

In [16]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,4.889500e+04,4.889500e+04,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,38843.000000,48895.000000,48895.000000
mean,1.901714e+07,6.762001e+07,40.728949,-73.952170,152.720687,7.029962,23.274466,1.373221,7.143982,112.781327
std,1.098311e+07,7.861097e+07,0.054530,0.046157,240.154170,20.510550,44.550582,1.680442,32.952519,131.622289
min,2.539000e+03,2.438000e+03,40.499790,-74.244420,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,9.471945e+06,7.822033e+06,40.690100,-73.983070,69.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,1.967728e+07,3.079382e+07,40.723070,-73.955680,106.000000,3.000000,5.000000,0.720000,1.000000,45.000000
75%,2.915218e+07,1.074344e+08,40.763115,-73.936275,175.000000,5.000000,24.000000,2.020000,2.000000,227.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,10000.000000,1250.000000,629.000000,58.500000,327.000000,365.000000


From this section of initial data exploration I have observed the following points:
1. name, neighbourhood group, last_review and reviews_per_month columns have NaN values.
2. There are no duplicate values in the dataset

**IMPUTATION**

In [17]:
df['name'].mode()

0    Hillside Hotel
Name: name, dtype: object

In [18]:
df['name']=df['name'].fillna("Hillside Hotel")

In [19]:
df.isnull().sum()

id                                    0
name                                  0
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [20]:
df['host_name'].mode()

0    Michael
Name: host_name, dtype: object

In [21]:
df['host_name']=df['host_name'].fillna("Michael")

In [22]:
df['last_review'].mode()

0    2019-06-23
Name: last_review, dtype: object

In [23]:
df['last_review']=df['last_review'].fillna("2019-06-23")

In [24]:
df['reviews_per_month'].mean()

1.3732214298586618

In [25]:
df['reviews_per_month']=df['reviews_per_month'].fillna(1.373)

In [26]:
df.isnull().sum()

id                                0
name                              0
host_id                           0
host_name                         0
neighbourhood_group               0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
last_review                       0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

1. name, neighbourhood_group, last_review and reviews_per_month column has several NaN values before this section
2. I imputed the NaN values in these columns by making use of statistical functions such as mean and mode

**DATE PARSING**

In [27]:
try:
  df['last_review'] = pd.to_datetime(df['last_review'], format='%d/%m/%Y')
except ValueError:
  df['last_review'] = pd.to_datetime(df['last_review'],errors='coerce')

In [28]:
df['last_review']

0       2018-10-19
1       2019-05-21
2       2019-06-23
3       2019-07-05
4       2018-11-19
           ...    
48890   2019-06-23
48891   2019-06-23
48892   2019-06-23
48893   2019-06-23
48894   2019-06-23
Name: last_review, Length: 48895, dtype: datetime64[ns]

As a precaution, I brought the dates contained in the last_review column into a common format

**OUTLIER DETECTION AND HANDLING**

In [29]:
df['z_latitude']=0.0

In [30]:
df['z_latitude']=zscore(df['latitude'])

In [31]:
condition_to_drop = (df['z_latitude'] > 3.0)
df = df[~condition_to_drop]
condition_to_drop = (df['z_latitude'] < -3.0)
df = df[~condition_to_drop]

In [32]:
df[(df['z_latitude'] < -3.0)]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,z_latitude


In [33]:
df[(df['z_latitude'] > 3.0)]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,z_latitude


In [34]:
df['z_longitude']=0.0

In [35]:
df['z_longitude']=zscore(df['longitude'])

In [36]:
df['z_longitude']

0       -0.444611
1       -0.694361
2        0.222924
3       -0.168352
4        0.177136
           ...   
48890    0.046565
48891    0.414180
48892    0.074607
48893   -0.855385
48894   -0.816170
Name: z_longitude, Length: 48796, dtype: float64

In [37]:
df.head(3)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,z_latitude,z_longitude
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.210,6,365,-1.493849,-0.444611
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.380,2,355,0.452436,-0.694361
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,2019-06-23,1.373,1,365,1.468399,0.222924


In [38]:
condition_to_drop=(df['z_longitude']<-3.0)
df=df[~condition_to_drop]
condition_to_drop=(df['z_longitude']>3.0)
df=df[~condition_to_drop]

In [39]:
df[df['z_longitude'] < -3.0]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,z_latitude,z_longitude


In [40]:
df[df['z_longitude'] > 3.0]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,z_latitude,z_longitude


In [41]:
df['z_price']=0

In [42]:
df['z_price']=zscore(df['price'])

In [43]:
df['z_price']

0       -0.020398
1        0.294504
2       -0.016254
3       -0.269004
4       -0.306295
           ...   
48890   -0.347730
48891   -0.472033
48892   -0.161275
48893   -0.409881
48894   -0.264861
Name: z_price, Length: 47546, dtype: float64

In [44]:
condition_to_be_checked=(df['z_price']>3.0)

In [45]:
df[df['z_price']>3.0]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,z_latitude,z_longitude,z_price
496,174966,Luxury 2Bed/2.5Bath Central Park View,836168,Henry,Manhattan,Upper West Side,40.77350,-73.98697,Entire home/apt,2000,30,30,2018-05-05,0.330,11,0,0.817009,-0.764467,7.649115
762,273190,6 Bedroom Landmark West Village Townhouse,605463,West Village,Manhattan,West Village,40.73301,-74.00268,Entire home/apt,1300,5,28,2018-09-25,0.310,4,297,0.074476,-1.108641,4.748705
946,363673,Beautiful 3 bedroom in Manhattan,256239,Tracey,Manhattan,Upper West Side,40.80142,-73.96931,Private room,3000,7,0,2019-06-23,1.373,1,365,1.329025,-0.377573,11.792557
1105,468613,$ (Phone number hidden by Airbnb) weeks - room f,2325861,Cynthia,Manhattan,Lower East Side,40.72152,-73.99279,Private room,1300,1,0,2019-06-23,1.373,1,0,-0.136236,-0.891971,4.748705
1414,634353,Luxury 1Bed with Central Park Views,836168,Henry,Manhattan,Upper West Side,40.77428,-73.98594,Entire home/apt,1000,30,44,2015-09-28,0.530,11,364,0.831313,-0.741902,3.505672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48301,36186719,Private Bedroom in the Heart of Chelsea!,268920555,Terrence Jake,Manhattan,Chelsea,40.74531,-73.99454,Private room,999,30,0,2019-06-23,1.373,1,270,0.300042,-0.930310,3.501528
48304,36189195,Next to Times Square/Javits/MSG! Amazing 1BR!,270214015,Rogelio,Manhattan,Hell's Kitchen,40.75533,-73.99866,Entire home/apt,2999,30,0,2019-06-23,1.373,1,222,0.483795,-1.020571,11.788414
48305,36189257,2BR Near Museum Mile! Upper East Side!,272166348,Mary Rotsen,Manhattan,Upper East Side,40.78132,-73.95262,Entire home/apt,1999,30,0,2019-06-23,1.373,1,270,0.960418,-0.011929,7.644971
48523,36308562,"Tasteful & Trendy Brooklyn Brownstone, near Train",217732163,Sandy,Brooklyn,Bedford-Stuyvesant,40.68767,-73.95805,Entire home/apt,1369,1,0,2019-06-23,1.373,1,349,-0.757001,-0.130889,5.034602


In [46]:
df[df['z_price']<-3.0]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,z_latitude,z_longitude,z_price


1. Outliers can cause problems in model building so I decided to remove them from the dataset
2. I utilized z score technique to detect outliers in latitude, longitude and price columns as they are the most crucial columns to the analysis.
3. If z score lies between -3 and +3 then the data point is not an outlier otherwise it is classified as an outlier.
4. I created columns of z_latitude and z_longitude to store the z-scores of latitude and longitude columns respectively.
5. I removed the rows containing z_latitude > 3 and z_latitude<-3
6. There were no rows for z_longitude<-3.
7. I removed the rows where z_longitude>3
8. I created columns of z_price to store the z-scores of price column .
9. Since most of the outliers detected through the price column  have names such as "luxury" and "trendy" and might have a reason for their high price, I will be keeping the outliers in the price columns as it is and not deal with them.
10. There were no rows for z_price<-3


**TEXT PRE PROCESSING**

In [47]:
def remove_stopwords(text):
    token_text = word_tokenize(text)
    filter_text = [word for word in token_text if word.lower() not in stopwords.words('english')]
    clean_text = ' '.join(filter_text)
    return clean_text

In [48]:
df['name'].head(2)

0    Clean & quiet apt home by the park
1                 Skylit Midtown Castle
Name: name, dtype: object

In [49]:
df['name']=df['name'].apply(remove_stopwords)

In [50]:
df['name'].head(2)

0    Clean & quiet apt home park
1          Skylit Midtown Castle
Name: name, dtype: object

In [51]:
punc=string.punctuation

In [52]:
punc

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [53]:
def remove_punctuations(text):
  clean_text=""
  for c in text:
    if c not in  punc:
      clean_text=clean_text+""+c
  return clean_text



In [54]:
df['name']=df['name'].apply(remove_punctuations)
df['host_name']=df['host_name'].apply(remove_punctuations)
df['neighbourhood_group']=df['neighbourhood_group'].apply(remove_punctuations)
df['neighbourhood']=df['neighbourhood'].apply(remove_punctuations)


In [55]:
df.head(2)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,z_latitude,z_longitude,z_price
0,2539,Clean quiet apt home park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365,-1.493849,-0.444611,-0.020398
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355,0.452436,-0.694361,0.294504


In [56]:
def to_lower_case(text):
  return text.lower()

In [57]:
df['name']=df['name'].apply(to_lower_case)
df['host_name']=df['host_name'].apply(to_lower_case)
df['neighbourhood_group']=df['neighbourhood_group'].apply(to_lower_case)
df['neighbourhood']=df['neighbourhood'].apply(to_lower_case)
df['room_type']=df['room_type'].apply(to_lower_case)

In [58]:
df.head(2)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,z_latitude,z_longitude,z_price
0,2539,clean quiet apt home park,2787,john,brooklyn,kensington,40.64749,-73.97237,private room,149,1,9,2018-10-19,0.21,6,365,-1.493849,-0.444611,-0.020398
1,2595,skylit midtown castle,2845,jennifer,manhattan,midtown,40.75362,-73.98377,entire home/apt,225,1,45,2019-05-21,0.38,2,355,0.452436,-0.694361,0.294504


1. In this section, I have dealth with cleaning the text based columns such as name, host_name, neighbourhood_group and neighbourhood.
2. I removed stopwords from the columns to concentrate on the more important words
3. I also included a punctuation removal function
4. I converted all these textual columns into lower case for easy handling.

**COLUMN REMOVAL**

In [59]:
df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'z_latitude', 'z_longitude', 'z_price'],
      dtype='object')

In [60]:
df=df.drop(['id'],axis=1)
df=df.drop(['host_name'],axis=1)
df=df.drop(['z_latitude'],axis=1)
df=df.drop(['z_longitude'],axis=1)
df=df.drop(['z_price'],axis=1)

In [61]:
df.columns

Index(['name', 'host_id', 'neighbourhood_group', 'neighbourhood', 'latitude',
       'longitude', 'room_type', 'price', 'minimum_nights',
       'number_of_reviews', 'last_review', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365'],
      dtype='object')

1. in this section, I removed some columns of id, host_name, z_latitude, z_longitude and z_price as they dont play a major role in the upcoming cleaning process and have less contribution in the visualization process.

**LABEL ENCODING**

In [62]:
le=LabelEncoder()

In [63]:
df['neighbourhood_group']=le.fit_transform(df['neighbourhood_group'])
df['neighbourhood']=le.fit_transform(df['neighbourhood'])
df['room_type']=le.fit_transform(df['room_type'])

In [64]:
df.head(2)

,name,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,clean quiet apt home park,2787,1,82,40.64749,-73.97237,1,149,1,9,2018-10-19,0.21,6,365
1,skylit midtown castle,2845,2,96,40.75362,-73.98377,0,225,1,45,2019-05-21,0.38,2,355


In [65]:
df.columns

Index(['name', 'host_id', 'neighbourhood_group', 'neighbourhood', 'latitude',
       'longitude', 'room_type', 'price', 'minimum_nights',
       'number_of_reviews', 'last_review', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365'],
      dtype='object')

1. In this section, I focussed on converting many of the text based columns into numeric type using label encoding method to help in easy handling.

**MIN MAX SCALING**

In [66]:
scaler=MinMaxScaler()

In [67]:
non_scaling_features=df.drop(['latitude','longitude','price', 'minimum_nights',
       'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365'],axis=1)
scaling_features=df.drop(['name', 'host_id', 'neighbourhood_group', 'neighbourhood','room_type','last_review'],axis=1)



In [68]:
non_scaling_features.to_numpy()


array([['clean  quiet apt home park', 2787, 1, 82, 1,
        Timestamp('2018-10-19 00:00:00')],
       ['skylit midtown castle', 2845, 2, 96, 0,
        Timestamp('2019-05-21 00:00:00')],
       ['village harlem  new york ', 4632, 2, 74, 1,
        Timestamp('2019-06-23 00:00:00')],
       ...,
       ['sunny studio historical neighborhood', 23492952, 2, 74, 0,
        Timestamp('2019-06-23 00:00:00')],
       ['43rd st time squarecozy single bed', 30985759, 2, 75, 2,
        Timestamp('2019-06-23 00:00:00')],
       ['trendy duplex heart hell s kitchen', 68119814, 2, 75, 1,
        Timestamp('2019-06-23 00:00:00')]], dtype=object)

In [69]:
scaling_features.to_numpy()

array([[ 4.064749e+01, -7.397237e+01,  1.490000e+02, ...,  2.100000e-01,
         6.000000e+00,  3.650000e+02],
       [ 4.075362e+01, -7.398377e+01,  2.250000e+02, ...,  3.800000e-01,
         2.000000e+00,  3.550000e+02],
       [ 4.080902e+01, -7.394190e+01,  1.500000e+02, ...,  1.373000e+00,
         1.000000e+00,  3.650000e+02],
       ...,
       [ 4.081475e+01, -7.394867e+01,  1.150000e+02, ...,  1.373000e+00,
         1.000000e+00,  2.700000e+01],
       [ 4.075751e+01, -7.399112e+01,  5.500000e+01, ...,  1.373000e+00,
         6.000000e+00,  2.000000e+00],
       [ 4.076404e+01, -7.398933e+01,  9.000000e+01, ...,  1.373000e+00,
         1.000000e+00,  2.300000e+01]])

In [70]:
scaled_features=scaler.fit_transform(scaling_features)

In [71]:
scaled_features

array([[0.25086394, 0.4258955 , 0.0149    , ..., 0.00341939, 0.01533742,
        1.        ],
       [0.57543044, 0.38426991, 0.0225    , ..., 0.00632587, 0.00306748,
        0.97260274],
       [0.74485458, 0.53715266, 0.015     , ..., 0.02330313, 0.        ,
        1.        ],
       ...,
       [0.76237805, 0.51243291, 0.0115    , ..., 0.02330313, 0.        ,
        0.0739726 ],
       [0.58732683, 0.35743236, 0.0055    , ..., 0.02330313, 0.01533742,
        0.00547945],
       [0.60729686, 0.36396831, 0.009     , ..., 0.02330313, 0.        ,
        0.0630137 ]])

In [72]:
df1=pd.DataFrame(scaled_features,columns=['latitude','longitude','price', 'minimum_nights',
       'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365'])

In [73]:
df1.head()

,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,0.250864,0.425895,0.0149,0.000000,0.014827,0.003419,0.015337,1.000000
1,0.575430,0.384270,0.0225,0.000000,0.074135,0.006326,0.003067,0.972603
2,0.744855,0.537153,0.0150,0.001601,0.000000,0.023303,0.000000,1.000000
3,0.366005,0.471939,0.0089,0.000000,0.444811,0.079159,0.000000,0.531507
4,0.712713,0.529521,0.0080,0.007206,0.014827,0.001539,0.000000,0.000000


In [74]:
df2=pd.DataFrame(non_scaling_features,columns=['name', 'host_id', 'neighbourhood_group', 'neighbourhood','room_type','last_review'])

In [75]:
df2.head()

,name,host_id,neighbourhood_group,neighbourhood,room_type,last_review
0,clean quiet apt home park,2787,1,82,1,2018-10-19
1,skylit midtown castle,2845,2,96,0,2019-05-21
2,village harlem new york,4632,2,74,1,2019-06-23
3,cozy entire floor brownstone,4869,1,30,0,2019-07-05
4,entire apt spacious studioloft central park,7192,2,49,0,2018-11-19


In [76]:
df=pd.concat([df2,df1],axis=1)

In [77]:
df.head(2)

,name,host_id,neighbourhood_group,neighbourhood,room_type,last_review,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,clean quiet apt home park,2787.0,1.0,82.0,1.0,2018-10-19,0.250864,0.425895,0.0149,0.0,0.014827,0.003419,0.015337,1.000000
1,skylit midtown castle,2845.0,2.0,96.0,0.0,2019-05-21,0.575430,0.384270,0.0225,0.0,0.074135,0.006326,0.003067,0.972603
